In [8]:
from langchain_core.tools import tool
from langchain_openai import ChatOpenAI
from langchain_core.documents import Document
from pprint import pprint

In [9]:
animal_facts = {
    "강아지": "강아지는 인간과 가장 친한 반려동물 중 하나입니다. 충성심이 강하고 활발합니다.",
    "고양이": "고양이는 독립적인 성격을 가진 반려동물로, 깨끗하고 조용한 성격을 가졌습니다.",
    "코끼리": "코끼리는 육상 동물 중 가장 큰 동물로, 매우 높은 지능과 사회성을 지닙니다.",
    "기린": "기린은 목이 매우 길며, 주로 아프리카 사바나에서 서식합니다."
}

In [ ]:
@tool
def search_animal(query: str) -> list[Document]:
    '''
    이름에 해당하는 동물 정보를 간단히 반환합니다.
    '''
    for key in animal_facts:
        if key in query:
            return [Document(page_content=animal_facts[key])]
    return [Document(page_content='해당 동물 정보를 찾을 수 없습니다.')]
        
    # if query in animal_facts:
    #     return [Document(page_content=animal_facts[query])]
    # return [Document(page_content='해당 동물 정보를 찾을 수 없습니다.')]

In [15]:
llm = ChatOpenAI(model='gpt-4o')
llm_with_tools = llm.bind_tools(tools=[search_animal])

In [16]:
question = '고양이에 대해 알려줘'
ai_msg = llm_with_tools.invoke(question)

pprint(ai_msg)
print('-'*100)
pprint(ai_msg.content)
print('-'*100)
pprint(ai_msg.tool_calls)
print('-'*100)

AIMessage(content='', additional_kwargs={'tool_calls': [{'id': 'call_px5bgIFkrZHg1shEmiPWab7G', 'function': {'arguments': '{"query":"고양이"}', 'name': 'search_animal'}, 'type': 'function'}], 'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 17, 'prompt_tokens': 53, 'total_tokens': 70, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 0, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}}, 'model_name': 'gpt-4o-2024-08-06', 'system_fingerprint': 'fp_cbf1785567', 'finish_reason': 'tool_calls', 'logprobs': None}, id='run--4c1a815e-19cb-40e1-854e-da15ee27cbe7-0', tool_calls=[{'name': 'search_animal', 'args': {'query': '고양이'}, 'id': 'call_px5bgIFkrZHg1shEmiPWab7G', 'type': 'tool_call'}], usage_metadata={'input_tokens': 53, 'output_tokens': 17, 'total_tokens': 70})
----------------------------------------------------------------------------------------------------
''
----

In [13]:
from datetime import datetime
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.runnables import chain

today = datetime.today().strftime('%Y-%m-%d')

prompt = ChatPromptTemplate([
    ('system', f"You are helpful AI assistant. Today's date is {today}."),
    ('human', '{user_input}'),
    ('placeholder', '{message}')
])

llm = ChatOpenAI(model='gpt-4o-mini')

llm_with_tools = llm.bind_tools(tools=[search_animal])

llm_chain = prompt | llm_with_tools

@chain
def web_search_chain(user_input: str):
    input_ = {'user_input': user_input}
    ai_msg = llm_chain.invoke(input_)

    tool_msgs = search_animal.batch(ai_msg.tool_calls)

    return llm_chain.invoke({**input_, 'message': [ai_msg, *tool_msgs]})

response = web_search_chain.invoke(question)

pprint(response.content)

'고양이는 독립적인 성격을 가진 반려동물로, 깨끗하고 조용한 성격을 가지고 있습니다.'
